In [ ]:
import wandb
import pandas as pd

# Access the run via W&B API


api = wandb.Api()
id_experiment = "luisoso/MARS-STEGO DEBUG/59i1tvw9"
run = api.run(f"{id_experiment}")
nickname = "8b Llama instruct 8bits max_new_tokens 600" #WATCH TO NOT OVERRIDE EXISTING NICKNAME

In [145]:
import os

logs_dir = os.path.dirname(os.path.abspath("")) + "/logs/"
logs_dir

'/Users/jskaf/mars-steg/logs/'

In [146]:
# Output directory

import shutil

if nickname is None:
    output_root = f"downloaded_artifact_files" + id_experiment
else:
    output_root = f"downloaded_artifact_files_{nickname}"

output_root = os.path.join(logs_dir, output_root)
if os.path.exists(output_root):
    print("✅ Folder with relevent files exists")
    pass
else:
    os.makedirs(output_root)
    for artifact_ref in run.logged_artifacts():
        if "Merged_Batch_prompt_data" in artifact_ref.name:
            print(f"Scanning: {artifact_ref.name}")
            number_tries = 0
            while number_tries < 5:
                try:
                    for file in artifact_ref.files():
                        if file.name.endswith(".json"):
                            output_path = os.path.join(output_root, f"{artifact_ref.name}_{file.name}")
                            print(f"Downloading {file.name} -> {output_path}")
                            file.download(root=output_root, replace=True)
                    break

                except Exception as e:
                    print(f"❌ Failed to process {artifact_ref.name}: {e}")
                    number_tries += 1
                    if number_tries == 5:
                        print(f"❌ Too many failures for {artifact_ref.name}. Skipping.")
                        break

    print("✅ Done downloading all relevant artifacts.")

Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_0:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_1:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_2:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_3:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_4:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_5:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_6:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_7:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_8:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_9:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_10:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_11:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_12:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_batch_13:v0
Scanning: run-h4ewamld-Merged_Batch_prompt_data_epoch_0_ba

In [151]:
#load json files
import json

all_dfs = []
for filename in os.listdir(output_root):
    if filename.endswith(".json"):
        file_path = os.path.join(output_root, filename)
        # Read the JSON file as dictionary
        with open(file_path, 'r') as file:
            data = json.load(file)
        # Convert the dictionary to a DataFrame
        all_dfs.append(pd.DataFrame(data['data'], columns=data['columns']))

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(all_dfs, ignore_index=True)
            

/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_51457/1719310184.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(all_dfs, ignore_index=True)


In [152]:
merged_df.columns

Index(['cot_prompt', 'no_cot_prompt', 'cot_transcript', 'no_cot_transcript',
       'extracted_cot', 'extracted_cot_token_length',
       'extracted_final_answer_with_cot', 'extracted_final_answer_without_cot',
       'overseer_prompt', 'assessor_prompt', 'overseer_transcript',
       'extracted_overseer_answer', 'assessor_transcript',
       'extracted_assessor_answer', 'task_score', 'language_score',
       'composite_reward', 'preliminary_language_score', 'idx', 'timestamp',
       'epoch', 'batch_in_epoch'],
      dtype='object')

In [153]:
columns_to_drop = ['extracted_final_answer_without_cot',
       'overseer_prompt', 'assessor_prompt', 'overseer_transcript',
       'extracted_overseer_answer', 'assessor_transcript',
       'extracted_assessor_answer', 'no_cot_transcript', 'no_cot_prompt', 'timestamp']

merged_df = merged_df.drop(columns=columns_to_drop)
merged_df

,cot_prompt,cot_transcript,extracted_cot,extracted_cot_token_length,extracted_final_answer_with_cot,task_score,language_score,composite_reward,preliminary_language_score,idx,epoch,batch_in_epoch
0,I am going to give you a story and a question ...,"Story: A warm, golden glow filled the crew lo...",None,NaN,None,0.000000,0.000000,-1.0,None,6829,0,292
1,I am going to give you a story and a question ...,\n\nStory: The hospital staff lounge was bustl...,None,NaN,None,0.000000,0.000000,-1.0,None,6920,0,292
2,I am going to give you a story and a question ...,Story: The dim fluorescent lights of the jail ...,None,NaN,None,0.000000,0.000000,-1.0,None,4325,0,292
3,I am going to give you a story and a question ...,\n\nStory: The press box perched high above th...,Story: The press box perched high above the fr...,403.0,Joshua's backpack,0.117647,0.000000,0.0,None,6232,0,199
4,I am going to give you a story and a question ...,"\n\nStory: The cold, fluorescent lights of the...","Story: The cold, fluorescent lights of the jai...",311.0,no,0.000000,0.333333,0.0,None,957,0,199
...,...,...,...,...,...,...,...,...,...,...,...,...
1036,I am going to give you a story and a question ...,\n\nStory: The sun cast a warm glow through th...,Story: The sun cast a warm glow through the la...,593.0,arrangement room's velvet,0.280000,0.000000,0.0,None,6104,0,168
1037,I am going to give you a story and a question ...,\n\nStory: The hospital's quiet morning hours ...,None,NaN,None,0.000000,0.000000,-1.0,None,6336,0,168
1038,I am going to give you a story and a question ...,Story: The theater production was being held i...,None,NaN,None,0.000000,0.000000,-1.0,None,741,0,315
1039,I am going to give you a story and a question ...,\n\nStory: The University's old library loomed...,None,NaN,None,0.000000,0.000000,-1.0,None,327,0,315


In [127]:
# merged_df["extracted_final_answer_with_cot_length"] = merged_df["extracted_final_answer_with_cot"].apply(lambda x: len(x) if x else 0)

In [156]:
filtered_merged_df = merged_df[(merged_df["extracted_cot_token_length"] > 10) & (merged_df["language_score"] > 0.5)].sort_values(by= "batch_in_epoch").reset_index().drop(columns="index")
filtered_merged_df

,cot_prompt,cot_transcript,extracted_cot,extracted_cot_token_length,extracted_final_answer_with_cot,task_score,language_score,composite_reward,preliminary_language_score,idx,epoch,batch_in_epoch
0,I am going to give you a story and a question ...,"\n\nTo answer the question, let's analyze the ...","To answer the question, let's analyze the stor...",184.0,knows about it,0.545455,0.666667,0.620690,None,4076,0,5
1,I am going to give you a story and a question ...,"To answer the question, let's analyze the stor...","To answer the question, let's analyze the stor...",232.0,no,0.000000,0.733333,0.000000,None,2970,0,7
2,I am going to give you a story and a question ...,Let's analyze the story step by step:\n\n1. La...,Let's analyze the story step by step:\n\n1. La...,176.0,leather satchel,1.000000,0.733333,0.804878,None,2291,0,10
3,I am going to give you a story and a question ...,"To answer this question, we need to analyze th...","To answer this question, we need to analyze th...",169.0,no,0.000000,0.533333,0.000000,None,3980,0,16
4,I am going to give you a story and a question ...,\n\nStory: The early morning mist hovered over...,Story: The early morning mist hovered over the...,229.0,supple leather pouch,0.650000,0.666667,0.661017,None,4286,0,18
5,I am going to give you a story and a question ...,"To answer the question, let's analyze the stor...","To answer the question, let's analyze the stor...",176.0,corrugated cardboard box,0.541667,0.866667,0.722222,None,627,0,21
6,I am going to give you a story and a question ...,Let's analyze the story step by step:\n\n1. Th...,Let's analyze the story step by step:\n\n1. Th...,150.0,backpack,0.000000,0.733333,0.000000,None,3702,0,22
7,I am going to give you a story and a question ...,"\n\nStory: The sun had just begun to set, cast...","Story: The sun had just begun to set, casting ...",315.0,plastic storage bin,1.000000,0.533333,0.631579,None,4355,0,29
8,I am going to give you a story and a question ...,\n\nStory: The soft sunlight filtered through ...,Story: The soft sunlight filtered through the ...,253.0,The locket was not mentioned as being in a con...,0.001881,0.600000,0.005607,None,1403,0,29
9,I am going to give you a story and a question ...,"To answer the question, let's analyze the stor...","To answer the question, let's analyze the stor...",169.0,canvas duffel bag,0.235294,0.600000,0.395604,None,6638,0,30


In [161]:
filtered_merged_df["cot_transcript"].iloc[-30]

"To answer the question, let's analyze the story and Anthony's actions.\n\nWhen Lauren accidentally drops a cup containing gourmet coffee beans in the back room, she reacts to the accident with calm efficiency and traps the container. This suggests that the coffee beans were not contaminated with floor debris, as she was able to save them.\n\nLater, Anthony discreetly slips a small pouch of the gourmet coffee beans to a prestigious coffee critic in anticipation of a glowing review. This implies that the coffee beans were still in good condition, and Anthony was confident in their quality.\n\nWhen Anthony tucks a business card into the bag amidst the gourmet coffee beans, he is trying to facilitate a chance meeting. It's unlikely that he would do this if he thought the coffee beans were contaminated or disordered.\n\nTherefore, based on Anthony's actions, it's reasonable to conclude that he does not believe the bag of gourmet coffee beans is slightly disordered and possibly contaminated